In [ ]:
# Add Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense,Dropout,Input # type: ignore
from sklearn.metrics import recall_score
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# Load the dataset
data = pd.read_csv('../data/bank-additional-full_normalised.csv')
# Check if all values are between 0 and 1
if not ((data >= 0) & (data <= 1)).all().all():
    print("Values outside the range [0, 1] found in the dataset.")
    print(data[(data < 0) | (data > 1)])
    
# Separate features and target
X = data.drop('class', axis=1)
y = data['class'].to_numpy()

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Standardize the data 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def create_model():
    # Define the DNN model
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

# Define the DNN model
model = create_model()

# Define a different DNN model for weighted training
model_weighted = create_model()

# Compile the models
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_weighted.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from sklearn.utils.class_weight import compute_class_weight
# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_test), y=y_test)
class_weights_dict  = {0 : class_weights[0], 1 : class_weights[1]}
print(class_weights_dict)

In [ ]:
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint

class F1ScoreCallback(Callback):
    def __init__(self, validation_data):
        super(F1ScoreCallback, self).__init__()
        self.validation_data = validation_data
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0])) > 0.5).astype(int)
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict, pos_label=1)
        print(f' — val_f1: {_val_f1:.4f}')
        if not hasattr(self, 'best_val_f1') or _val_f1 > self.best_val_f1:
            self.best_val_f1 = _val_f1
            self.best_weights = self.model.get_weights()
    

early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)


In [ ]:
# Train the model

history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=150,
    validation_split=0.2,
    callbacks=[early_stopping,F1ScoreCallback((X_test, y_test))]
    # class_weight=class_weights_dict
)

In [ ]:

# Train the weighted model
history_weighted = model_weighted.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=150,
    validation_split=0.2,
    callbacks=[early_stopping,F1ScoreCallback((X_test, y_test))],
    class_weight=class_weights_dict
)


In [ ]:
from sklearn.metrics import classification_report

# Evaluate the model
# Evaluate the unweighted model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Unweighted Model Test Accuracy: {accuracy:.4f}')

# Get predictions for the unweighted model
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # Convert probabilities to binary classes

# Generate classification report for the unweighted model
report = classification_report(y_test, y_pred_classes)
print('Unweighted Model Classification Report:')
print(report)

# Evaluate the weighted model
loss_weighted, accuracy_weighted = model_weighted.evaluate(X_test, y_test)
print(f'Weighted Model Test Accuracy: {accuracy_weighted:.4f}')

# Get predictions for the weighted model
y_pred_weighted = model_weighted.predict(X_test)
y_pred_classes_weighted = (y_pred_weighted > 0.5).astype(int)  # Convert probabilities to binary classes

# Generate classification report for the weighted model
report_weighted = classification_report(y_test, y_pred_classes_weighted)
print('Weighted Model Classification Report:')
print(report_weighted)



In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values for unweighted model
plt.figure(figsize=(12, 6))

plt.subplot(2, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Unweighted Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(2, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Unweighted Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation accuracy values for weighted model
plt.subplot(2, 2, 3)
plt.plot(history_weighted.history['accuracy'])
plt.plot(history_weighted.history['val_accuracy'])
plt.title('Weighted Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(2, 2, 4)
plt.plot(history_weighted.history['loss'])
plt.plot(history_weighted.history['val_loss'])
plt.title('Weighted Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()



In [ ]:
model_dir = 'dnn_models'

# Save the unweighted model
model.save(f'{model_dir}/unweighted_model.keras')

# Save the weighted model
model_weighted.save(f'{model_dir}/weighted_model.keras')

In [ ]:
# Load the unweighted model
loaded_model = load_model(f'{model_dir}/unweighted_model.keras')

# Load the weighted model
loaded_model_weighted = load_model(f'{model_dir}/weighted_model.keras')

# Evaluate the loaded unweighted model
loss_loaded, accuracy_loaded = loaded_model.evaluate(X_test, y_test)
print(f'Loaded Unweighted Model Test Accuracy: {accuracy_loaded:.4f}')

# Get predictions for the loaded unweighted model
y_pred_loaded = loaded_model.predict(X_test)
y_pred_classes_loaded = (y_pred_loaded > 0.5).astype(int)  # Convert probabilities to binary classes

# Generate classification report for the loaded unweighted model
report_loaded = classification_report(y_test, y_pred_classes_loaded)
print('Loaded Unweighted Model Classification Report:')
print(report_loaded)

# Evaluate the loaded weighted model
loss_loaded_weighted, accuracy_loaded_weighted = loaded_model_weighted.evaluate(X_test, y_test)
print(f'Loaded Weighted Model Test Accuracy: {accuracy_loaded_weighted:.4f}')

# Get predictions for the loaded weighted model
y_pred_loaded_weighted = loaded_model_weighted.predict(X_test)
y_pred_classes_loaded_weighted = (y_pred_loaded_weighted > 0.5).astype(int)  # Convert probabilities to binary classes

# Generate classification report for the loaded weighted model
report_loaded_weighted = classification_report(y_test, y_pred_classes_loaded_weighted)
print('Loaded Weighted Model Classification Report:')
print(report_loaded_weighted)